# lstm 주식종가예측
KOSPI-200과 KOSDAQ-150의 주어진 기간에 마지막으로 거래된 가격(종가)를 예측

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import FinanceDataReader as fdr
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import datetime
from tqdm import tqdm
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM,Dropout,Bidirectional
from keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM,Dropout,Bidirectional
from keras.callbacks import EarlyStopping
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller

In [3]:
df=pd.read_csv("stock_list.csv")
df['종목코드'] = df['종목코드'].apply(lambda x : str(x).zfill(6))

In [2]:
import keras.backend as K

In [11]:
result4=pd.read_csv('sample_submission.csv')

In [12]:
for code in tqdm(df['종목코드'].values):
    start_date = '20191101'
    end_date = '20211031'
    sample_code = code
    stock1 = fdr.DataReader(sample_code, start = start_date, end = end_date)
    stock=stock1[['Close']]
    sc = MinMaxScaler()
    stock=sc.fit_transform(stock)
    df1=pd.DataFrame(stock, columns=['Close'],index=stock1.index)
    dfpred = df1.iloc[:-15,]  # 
    dfreal = df1.iloc[-15:,]
    dfpred=dfpred[['Close']]  # train
    dfreal=dfreal[['Close']]  # test
    for s in range(1, 11): # lag 추가
        dfpred['lag_{}'.format(s)] = dfpred['Close'].shift(s)
        dfreal['lag_{}'.format(s)] = dfreal['Close'].shift(s)
    x_train = dfpred.dropna().drop('Close', axis=1)
    y_train = dfpred.dropna()[['Close']]
    x_test = dfreal.dropna().drop('Close', axis=1)
    y_test = dfreal.dropna()[['Close']]
    x_trian=x_train.values
    y_train=y_train.values
    x_test=x_test.values
    y_test=y_test.values
    x_train=np.asarray(x_train)
    x_test=np.asarray(x_test)
    y_train=np.asarray(y_train)
    y_test=np.asarray(y_test) 
    x_train_t = x_train.reshape(x_train.shape[0],10,1)
    x_test_t = x_test.reshape(x_test.shape[0],10,1)
    K.clear_session()
    model = Sequential()
    model.add(LSTM( 10, input_shape=(10,1))) # timestep, feature
    model.add(Dense(5))
    model.compile(loss='mean_squared_error', optimizer='adam')
    early_stop=EarlyStopping(monitor='loss', patience=7, verbose=1 )
    model.fit(x_train_t,y_train,epochs=500,batch_size=11,verbose=1,callbacks=[early_stop])
    y_hat = model.predict(x_test_t)
    y_hat = { 'Close' : y_hat.sum() }
    df1=df1.append(y_hat, ignore_index=True)
    df1=sc.inverse_transform(df1)  
    df1=df1[-6:]
    for x in range(0,371): # 확인해보기
        if code==result4.columns[x]:
            temp=x
            break
    for k in range(0,5):
        result4.iloc[k,[temp]]=df1[k]

  0%|                                                                                          | 0/370 [00:00<?, ?it/s]

Epoch 1/500
43/43 [==============================] - 2s 3ms/step - loss: 0.1631
Epoch 2/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0365
Epoch 3/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0110
Epoch 4/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0048
Epoch 5/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0031
Epoch 6/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0027
Epoch 7/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 8/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 9/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 10/500
43/43 [==============================] - 0s 4ms/step - loss: 0.0024
Epoch 11/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0024
Epoch 12/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0023
Epoch 13/500
43/43 [=================

  0%|▏                                                                               | 1/370 [00:18<1:51:19, 18.10s/it]

Epoch 1/500
43/43 [==============================] - 2s 3ms/step - loss: 0.1825
Epoch 2/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0743
Epoch 3/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0202
Epoch 4/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0087
Epoch 5/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0061
Epoch 6/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0052
Epoch 7/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0048
Epoch 8/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0045
Epoch 9/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0042
Epoch 10/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0041
Epoch 11/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0040
Epoch 12/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0040
Epoch 13/500
43/43 [=================

  1%|▍                                                                               | 2/370 [00:33<1:41:34, 16.56s/it]

...

 99%|█████████████████████████████████████████████████████████████████████████████▌| 368/370 [1:21:09<00:24, 12.02s/it]

Epoch 1/500
43/43 [==============================] - 2s 3ms/step - loss: 0.1256
Epoch 2/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0396
Epoch 3/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0167
Epoch 4/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0074
Epoch 5/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0042
Epoch 6/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0032
Epoch 7/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0028
Epoch 8/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 9/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 10/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0024
Epoch 11/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0024
Epoch 12/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0023
Epoch 13/500
43/43 [=================

100%|█████████████████████████████████████████████████████████████████████████████▊| 369/370 [1:21:22<00:12, 12.41s/it]

Epoch 1/500
43/43 [==============================] - 2s 3ms/step - loss: 0.1211
Epoch 2/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0414
Epoch 3/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0145
Epoch 4/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0069
Epoch 5/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0046
Epoch 6/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0036
Epoch 7/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0030
Epoch 8/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0028
Epoch 9/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 10/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 11/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 12/500
43/43 [==============================] - 0s 3ms/step - loss: 0.0024
Epoch 13/500
43/43 [=================

100%|██████████████████████████████████████████████████████████████████████████████| 370/370 [1:21:36<00:00, 13.23s/it]


In [13]:
result4

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,28850.0,37200.0,61200.0,150000.0,106000.0,17250.0,48550.0,84300.0,100000.0,...,46500.0,50900.0,83300.0,36150.0,26600.0,51000.0,326000.0,253300.0,27350.0,17450.0
1,2021-11-02,29100.0,36900.0,62000.0,149000.0,109000.0,17350.0,50400.0,84800.0,102000.0,...,46550.0,54500.0,83100.0,37650.0,28150.0,52500.0,342000.0,259200.0,27700.0,17750.0
2,2021-11-03,29500.0,35750.0,61600.0,148000.0,108500.0,17300.0,49200.0,85000.0,101500.0,...,46400.0,54900.0,81800.0,38950.0,27850.0,53700.0,348500.0,264200.0,27500.0,17750.0
3,2021-11-04,29100.0,35300.0,60800.0,145500.0,102500.0,16650.0,49000.0,86000.0,106500.0,...,49200.0,50600.0,80600.0,37000.0,26250.0,54800.0,333500.0,263800.0,27500.0,17200.0
4,2021-11-05,27700.0,34800.0,59900.0,142000.0,103000.0,16700.0,47050.0,85100.0,103000.0,...,49100.0,52200.0,82300.0,36100.0,25950.0,53100.0,335000.0,261600.0,27150.0,17100.0
5,2021-11-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2021-11-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2021-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2021-12-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2021-12-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
